#### Huzaifa Kapasi- MIT License

# BigData Ingestion and Cleanup

In [3]:

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
import datetime

sqlContext = SQLContext(sc)
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[5]').appName('Data ingestion & cleanup').getOrCreate()

### Read the Trip Data from CSV
#### http://dx.doi.org/10.13012/J8PN93H8 

In [4]:

tripData = spark.read.csv(path='H:/Training/PySpark/data/trip_data/trip_data_1.csv', header=True, inferSchema=True)

### Read Fare Dataframe from CSV

In [5]:
fareData = spark.read.csv(path='H:/Training/PySpark/data/trip_fare/trip_fare_1.csv', header=True, inferSchema=True)

### CHECK SCHEMA OF TRIP AND FARE TABLES

In [5]:
tripData.printSchema()
fareData.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)

root
 |-- medallion: string (nullable = true)
 |--  hack_license: string (nullable = true)
 |--  vendor_id: string (nullable = true)
 |--  pickup_datetime: timestamp (nullable = true)
 |--  payment_type: string (nullable = true)
 |--  fare_amount: double (nullable = true)
 |--  surcharge: double (nullable = true)
 |--  mta_tax: double (nu

In [6]:
type(tripData)

pyspark.sql.dataframe.DataFrame

In [7]:
type(fareData)

pyspark.sql.dataframe.DataFrame

### Join Two Dataframes through Common Column Name

In [6]:
new_df = tripData.join(fareData, on=['medallion'], how='left_outer')

In [9]:
new_df.show()

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|        hack_license| vendor_id|    pickup_datetime| payment_type| fare_amount| surcharge| mta_tax| tip_amount| tolls_amount| total_amount|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------

### Drop some columns from the dataframe

In [7]:
taxi_df_train_drop = new_df.drop('medallion').drop('hack_license').drop('total_amount').drop('tolls_amount')

taxi_df_train_drop.show()

### Filter Column Data with preconditions 

In [10]:
 taxi_df_train__filter=taxi_df_train_drop.filter("passenger_count > 0 and passenger_count < 8 AND trip_distance > 0 AND trip_distance < 100 AND \
            trip_time_in_secs > 30 AND trip_time_in_secs < 7200")

In [11]:
taxi_df_train__filter.show()

+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|        hack_license| vendor_id|    pickup_datetime| payment_type| fare_amount| surcharge| mta_tax| tip_amount| tolls_amount| total_amount|
+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------